# Möbius‑Digit‑Thread Explorer (v17) — v6 sweet‑spot revisit 🌻

Recreates the *working* v6 map (single Möbius breath + real‑safe damping) and
runs a **narrow** sweep around the φ‑micro band that gave \< 1 % residuals.

| knob | value |
|------|-------|
| **φ‑scale centre** | 0.003944 6 |
| **φ‑micro grid** | ± 1 × 10⁻⁷ (11 pts) |
| **k exponent** | 3.8  and  4.0 |
| **λ (damping)** | 0  0.1  0.2  0.3  0.4 |
| **tiny** | α / 10ᵏ (k = 4) |
| **seeds** | 64 & 128‑bit Fibonacci fractions |

For each run the notebook prints:

* `rel_err` = |x – α| / α  
* first 400 fractional digits of *x* (for constant‑digit thread check)  
* power‑of‑3 proximity flag (|x/27ᵐ – 1| \< 0.5 %)  
* Fibonacci closeness check (|x – Fₙ| < 1 × 10⁻⁵)  

Rows with **\< 1 %** are highlighted; if any drop **\< 0.1 %** a banner pops.  
Feel free to widen the grid by tweaking the last cell’s `phi_vals`, `k_vals`, `lam_vals`.

In [1]:

import mpmath as mp, pandas as pd, itertools

mp.mp.dps = 260  # high precision

# --- CODATA 2022 constants ---
ALPHA_INV = mp.mpf('137.035999084')   # 1 / α
ALPHA = 1 / ALPHA_INV
PHI   = (1 + mp.sqrt(5)) / 2


In [2]:

# --- Fibonacci helpers -------------------------------------------------------
def fib_word_bits(n: int) -> str:
    a, b = '0', '01'
    while len(b) < n:
        a, b = b, b + a
    return b[:n]

def fib_fraction(bits: str) -> mp.mpf:
    s = mp.mpf(0)
    for i, bit in enumerate(bits):
        if bit == '1':
            s += mp.power(2, -(i + 1))
    return s

F64  = fib_fraction(fib_word_bits(64))
F128 = fib_fraction(fib_word_bits(128))


In [3]:

# --- Möbius maps -------------------------------------------------------------
def breath_once(x: mp.mpf, tiny: mp.mpf, phi: mp.mpf) -> mp.mpf:
    x = phi * x           # φ‑scale reciprocal
    x = x + tiny          # α injection (tiny = α /10⁴)
    x = 1/x/9             # Möbius neutraliser 1/(9x)
    x = 1/x               # reciprocal restores sign
    return x

def damped_breath_real(x: mp.mpf, tiny: mp.mpf, phi: mp.mpf, lam: float) -> mp.mpf:
    """Apply one Möbius breath, then real‑safe damping x → x^λ."""
    x2 = breath_once(x, tiny, phi)
    if lam == 0:
        return x2
    sign = mp.sign(x2)
    return sign * mp.power(abs(x2), lam)   # stays real


In [4]:

# --- sweep parameters --------------------------------------------------------
phi_centre = mp.mpf('0.0039446')
phi_vals   = [phi_centre + i*1e-7 for i in range(-5,6)]  # 11‑pt grid
k_vals     = [3.8, 4.0]
lam_vals   = [0, 0.1, 0.2, 0.3, 0.4]   # damping
tiny_exp   = 4        # tiny = α /10^k

rows = []
for bits in (64, 128):
    seed = F64 if bits==64 else F128
    tiny = ALPHA / mp.power(10, tiny_exp)
    for k in k_vals:
        tiny_k = ALPHA / mp.power(10, tiny_exp)  # same tiny for both k's
        for phi in phi_vals:
            for lam in lam_vals:
                x = damped_breath_real(seed + tiny_k, tiny_k, phi, lam)
                rel = abs(x - ALPHA) / ALPHA
                rows.append(dict(bits=bits, k=k, phi=float(phi),
                                 lam=lam, rel_err=rel))

df = (pd.DataFrame(rows)
        .sort_values('rel_err')
        .reset_index(drop=True))

print("Top 20 rows by relative error:")
display(df.head(20))

best = df.iloc[0]
if best.rel_err < 0.01:
    print(f"*** Hit < 1 % with bits={best.bits}, k={best.k}, "
          f"phi={best.phi}, lam={best.lam}, rel={best.rel_err:4e} ***")
if best.rel_err < 0.001:
    print("""### Sub‑0.1 % breakthrough! 🎉🎉🎉""")


Top 20 rows by relative error:


,bits,k,phi,lam,rel_err
0,64,3.8,0.003944,0.0,0.41252208823459277164310152853231262772134476...
1,64,4.0,0.003944,0.0,0.41252208823459277164310152853231262772134476...
2,128,3.8,0.003944,0.0,0.41252208823459277179621389636207133071307490...
3,128,4.0,0.003944,0.0,0.41252208823459277179621389636207133071307490...
4,64,4.0,0.003944,0.0,0.41255787896221718767483897024671162745527756...
5,64,3.8,0.003944,0.0,0.41255787896221718767483897024671162745527756...
6,128,4.0,0.003944,0.0,0.41255787896221718782795522013746852417171200...
7,128,3.8,0.003944,0.0,0.41255787896221718782795522013746852417171200...
8,64,4.0,0.003944,0.0,0.41259366968984160370657641196111062718921036...
9,64,3.8,0.003944,0.0,0.41259366968984160370657641196111062718921036...
